<a href="https://colab.research.google.com/github/Luv4as/projeto11_RL/blob/main/Projeto11_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dyna-Q com Prioritized Sweeping

# Instruções

### Pergunta:
- Qual o benefício de acrescentar “prioritized sweeping” (varredura priorizada) no Dyna-Q?

### Detalhes:
- Ver seção 8.4 do livro
- Explicar, usando um ambiente grid, como o Dyna-Q pode fazer muitas atualizações “ruins” (no planejamento ou indirect RL)
- Explicar a ideia de prioritized sweeping
- Implementar uns dois esquemas de prioritized sweeping
- Fazer experimentos com vários ambientes determinísticos
- Avaliar em termos de:
 - “tempo” por steps
 - tempo real em máquina de 1 processador (wallclock time)
 - quantidade de updates
- Experimentos (opções alternativas):
 - Comparar nos ambientes determinísticos (Taxi, Cliff, Racetrack, etc)
 - Labirintos de diferentes tamanhos, para reproduzir a figura abaixo (do livro)


# Imports gerais

In [ ]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # for saving videos
    !apt-get install ffmpeg

    !pip install gymnasium moviepy
    !pip install optuna

    # clone repository
    !git clone https://github.com/pablo-sampaio/rl_facil
    sys.path.append("/content/rl_facil")

else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

clear_output()

In [ ]:
import random as rand
import gymnasium as gym
import numpy as np
from queue import PriorityQueue
import matplotlib.pyplot as plt

import optuna

from util.experiments import repeated_exec
from util.plot import plot_result, plot_multiple_results
from util.notebook import display_videos_from_path
from util.qtable_helper import evaluate_qtable_policy, record_video_qtable

In [ ]:
# basta importar o módulo que o ambiente "RaceTrack-v0" é registrado no gym
import envs

# Dyna-Q


> Por que o Dyna-Q pode fazer muitas atualizações "ruins"?
- **Atualizações Uniformes:** O Dyna-Q realiza atualizações simuladas uniformemente para cada **estado** e **ação**, sem considerar a importância ou prioridade dessas atualizaçãoes.
- **Ineficiente em Espaços Grandes**: Em espaços grandes, onde os estados e ações são numerosos (```CliffWalking```, por exemplo), o Dyna-Q pode se tornar ineficiente porque tenta simular e atualizar todos de forma uniforme, independentemente de sua relevância para o problema atual.
- **Velocidade de Convergência**: Sem foco em atualizações mais significativas, o Dyna-Q pode demorar mais para alcançar uma política ótima ou quase ótima, especialmente em ambientes onde alguns estados têm um impacto muito maior nas recompensas cumulativas do que outros.

In [ ]:
def planning(model, planning_steps, Q, lr, gamma):
    all_s_a = list(model.keys())
    if len(all_s_a) < planning_steps:
        samples = rand.choices(all_s_a, k=planning_steps)
    else:
        samples = rand.sample(all_s_a, k=planning_steps)

    for s, a in samples:
        r, next_s, is_terminal = model[(s,a)]
        if is_terminal:
            V_next_s = 0
        else:
            V_next_s = np.max(Q[next_s])
        delta = (r + gamma * V_next_s) - Q[s,a]
        Q[s,a] = Q[s,a] + lr * delta

In [ ]:
# Esta é a política. Neste caso, escolhe uma ação com base nos valores da tabela Q, usando uma estratégia epsilon-greedy,
# dividindo a probabilidade igualmente em caso de empates entre ações de valor máximo.
from util.qtable_helper import epsilon_greedy

## Execution
Os ambientes utilizados para treinamento são:
- Taxi-v3
- FrozenLake-v1
- CliffWalking

In [ ]:
# Algoritmo Dyna Q
def run_dyna_q(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, planning_steps=5, verbose=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

    # inicializa a tabela Q
    Q = np.random.uniform(low=-0.01, high=+0.01, size=(env.observation_space.n, num_actions))

    # inicializa o modelo do ambiente
    model = dict()

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0

        state, _ = env.reset()

        # executa 1 episódio completo, fazendo atualizações na Q-table
        while not done:

            # escolhe a próxima ação -- usa epsilon-greedy
            action = epsilon_greedy(Q, state, epsilon)

            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated

            if terminated:
                # para estados terminais
                V_next_state = 0
            else:
                # para estados não-terminais -- valor máximo (melhor ação)
                V_next_state = np.max(Q[next_state])

            # atualiza a Q-table / direct RL
            delta = (reward + gamma * V_next_state) - Q[state,action]
            Q[state,action] = Q[state,action] + lr * delta

            # atualiza o modelo
            model[state,action] = (reward, next_state, terminated)

            # planejamento / indirect RL
            planning(model, planning_steps, Q, lr, gamma)

            sum_rewards += reward
            state = next_state

        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso
        if verbose and ((i+1) % 100 == 0):
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

    state = env.reset()
    reward = 0

    return sum_rewards_per_ep, Q

In [ ]:
# escolha o ambiente descomentando uma das linhas abaixo
ENV_NAME = "Taxi-v3"
#ENV_NAME = "WindyGrid-v0"
#ENV_NAME = "RaceTrack-v0"

LR = 0.3
GAMMA = 0.90
EPSILON = 0.1

#VERBOSE = True

In [ ]:
env = gym.make(ENV_NAME)

rmax = 10.0
EPISODES = 700

LR = 0.3
GAMMA = 0.90
EPSILON = 0.1

rewards2, qtable2 = run_dyna_q(env, EPISODES, LR, GAMMA, EPSILON, planning_steps=10, verbose=True)
print("Últimos resultados: media =", np.mean(rewards2[-20:]), ", desvio padrao =", np.std(rewards2[-20:]))

In [ ]:
# Mostra um gráfico de passos x retornos não descontados acumulados
plot_result(rewards2, rmax, cumulative='no', window=30)

In [ ]:
evaluate_qtable_policy(env, qtable2, 10, verbose=True);

In [ ]:
record_video_qtable(ENV_NAME, qtable2, episodes=3, folder='./videos-dynaq')

In [ ]:
display_videos_from_path("./videos-dynaq", speed=0.5)

# Dyna-Q com Prioritized Sweeping

 - **Atualizações Prioritárias**: Prioritized Sweeping foca nos estados e ações que mais precisam de atualização, baseando-se em um critério de prioridade como o erro de predição. Isso torna o algoritmo mais eficiente porque direciona o esforço computacional para onde ele é mais necessário.
 - **Convergência Rápida**: Ao realizar atualizações em estados e ações com grandes diferenças de predição, o aprendizado é acelerado. O agente ajusta rapidamente suas políticas e valores de estado para refletir as partes mais críticas do espaço de estados.
 - **Eficiência em Grandes Espaços**: Em ambientes com grandes estados de espaço, Prioritized Sweeping permite que o agente ignore áreas menos relevantes, reduzindo significativamente o tempo necessário para alcançar convergência.
 - **Adaptabilidade**: A abordagem de priorização permite que o agente se adapte mais prontamente a mudanças no ambiente ou na dinâmica do modelo, fazendo uso produtivo das informações recentes mais impactantes.

In [ ]:
# Parameters

ROWS = 6
COLS = 9

S = (2, 0)
G = (0, 8)

BLOCKS = [(1, 2), (2, 2), (3, 2), (0, 7), (1, 7), (2, 7), (4, 5)]

ACTIONS = ["left", "up", "right", "down"]

In [ ]:
class Maze:

    def __init__(self):
        self.rows = ROWS
        self.cols = COLS
        self.start = S
        self.goal = G
        self.blocks = BLOCKS
        self.state = S
        self.end = False

        # init maze
        self.maze = np.zeros((self.rows, self.cols))
        for b in self.blocks:
            self.maze[b] = -1

    def nxtPosition(self, action):
        r, c = self.state
        if action == "left":
            c -= 1
        elif action == "right":
            c += 1
        elif action == "up":
            r -= 1
        else:
            r += 1

        if (r >= 0 and r <= self.rows - 1) and (c >= 0 and c <= self.cols - 1):
            if (r, c) not in self.blocks:
                self.state = (r, c)
        return self.state

    def giveReward(self):
        if self.state == self.goal:
            self.end = True
            return 1
        else:
            return 0

    def showMaze(self):
        self.maze[self.state] = 1
        for i in range(0, self.rows):
            print('-------------------------------------')
            out = '| '
            for j in range(0, self.cols):
                if self.maze[i, j] == 1:
                    token = '*'
                if self.maze[i, j] == -1:
                    token = 'z'
                if self.maze[i, j] == 0:
                    token = '0'
                out += token + ' | '
            print(out)
        print('-------------------------------------')

In [ ]:
class PriorityAgent:

    def __init__(self, exp_rate=0.3, lr=0.1, n_steps=5, episodes=10, theta=0):
        self.maze = Maze()
        self.state = S
        self.actions = ACTIONS
        self.state_actions = []  # state & action track
        self.exp_rate = exp_rate
        self.lr = lr

        self.steps = n_steps
        self.episodes = episodes  # number of episodes going to play
        self.steps_per_episode = []

        self.Q_values = {}
        # model function
        self.model = {}
        for row in range(ROWS):
            for col in range(COLS):
                self.Q_values[(row, col)] = {}
                for a in self.actions:
                    self.Q_values[(row, col)][a] = 0

        # for priority sweeping
        self.theta = theta
        self.queue = PriorityQueue()
        self.predecessors = {}  # nxtState -> list[(curState, Action)...]

    def chooseAction(self):
        # epsilon-greedy
        mx_nxt_reward = -999
        action = ""

        if np.random.uniform(0, 1) <= self.exp_rate:
            action = np.random.choice(self.actions)
        else:
            # greedy action
            current_position = self.state
            # if all actions have same value, then select randomly
            if len(set(self.Q_values[current_position].values())) == 1:
                action = np.random.choice(self.actions)
            else:
                for a in self.actions:
                    nxt_reward = self.Q_values[current_position][a]
                    if nxt_reward >= mx_nxt_reward:
                        action = a
                        mx_nxt_reward = nxt_reward
        return action

    def reset(self):
        self.maze = Maze()
        self.state = S
        self.state_actions = []

    def play(self):
        for ep in range(self.episodes):
            while not self.maze.end:

                action = self.chooseAction()
                self.state_actions.append((self.state, action))

                nxtState = self.maze.nxtPosition(action)
                reward = self.maze.giveReward()

                # update priority queue
                tmp_diff = reward + np.max(list(self.Q_values[nxtState].values())) - self.Q_values[self.state][action]
                if tmp_diff > self.theta:
                    self.queue.put((-tmp_diff, (self.state, action)))  # -diff -> (state, action) pop the smallest

                # update model & predecessors
                if self.state not in self.model.keys():
                    self.model[self.state] = {}
                self.model[self.state][action] = (reward, nxtState)
                if nxtState not in self.predecessors.keys():
                    self.predecessors[nxtState] = [(self.state, action)]
                else:
                    self.predecessors[nxtState].append((self.state, action))
                self.state = nxtState

                # loop n times to randomly update Q-value
                for _ in range(self.steps):
                    if self.queue.empty():
                        break
                    _state, _action = self.queue.get()[1]
                    _reward, _nxtState = self.model[_state][_action]
                    self.Q_values[_state][_action] += self.lr * (_reward + np.max(list(self.Q_values[_nxtState].values())) - self.Q_values[_state][_action])

                    # loop for all state, action predicted lead to _state
                    if _state not in self.predecessors.keys():
                        continue
                    pre_state_action_list = self.predecessors[_state]

                    for (pre_state, pre_action) in pre_state_action_list:
                        pre_reward, _ = self.model[pre_state][pre_action]
                        pre_tmp_diff = pre_reward + np.max(list(self.Q_values[_state].values())) - self.Q_values[pre_state][pre_action]
                        if pre_tmp_diff > self.theta:
                            self.queue.put((-pre_tmp_diff, (pre_state, pre_action)))
            # end of game
            if ep % 10 == 0:
                print("episode", ep)
            self.steps_per_episode.append(len(self.state_actions))
            self.reset()


if __name__ == "__main__":
    pa = PriorityAgent()
    pa.play()

episode 0


In [ ]:
# Parameters
ROWS = 6
COLS = 9

S = (2, 0)
G = (0, 8)

BLOCKS = [(1, 2), (2, 2), (3, 2), (0, 7), (1, 7), (2, 7), (4, 5)]

ACTIONS = ["left", "up", "right", "down"]

In [ ]:
# Ambiente de labirinto
class Maze:

    def __init__(self):
        self.rows = ROWS
        self.cols = COLS
        self.start = S
        self.goal = G
        self.blocks = BLOCKS
        self.state = S
        self.end = False
        # init maze
        self.maze = np.zeros((self.rows, self.cols))
        for b in self.blocks:
            self.maze[b] = -1

    def nxtPosition(self, action):
        r, c = self.state
        if action == "left":
            c -= 1
        elif action == "right":
            c += 1
        elif action == "up":
            r -= 1
        else:
            r += 1

        if (r >= 0 and r <= self.rows-1) and (c >= 0 and c <= self.cols-1):
            if (r, c) not in self.blocks:
                self.state = (r, c)
        return self.state

    def giveReward(self):
        if self.state == self.goal:
            self.end = True
            return 1
        else:
            return 0

    def showMaze(self):
        self.maze[self.state] = 1
        for i in range(0, self.rows):
            print('-------------------------------------')
            out = '| '
            for j in range(0, self.cols):
                if self.maze[i, j] == 1:
                    token = '*'
                if self.maze[i, j] == -1:
                    token = 'z'
                if self.maze[i, j] == 0:
                    token = '0'
                out += token + ' | '
            print(out)
        print('-------------------------------------')

In [ ]:
class PriorityAgent:

    def __init__(self, exp_rate=0.3, lr=0.1, n_steps=5, episodes=1, theta=0):
        self.maze = Maze()
        self.state = S
        self.actions = ACTIONS
        self.state_actions = []  # state & action track
        self.exp_rate = exp_rate
        self.lr = lr

        self.steps = n_steps
        self.episodes = episodes  # number of episodes going to play
        self.steps_per_episode = []

        self.Q_values = {}
        # model function
        self.model = {}
        for row in range(ROWS):
            for col in range(COLS):
                self.Q_values[(row, col)] = {}
                for a in self.actions:
                    self.Q_values[(row, col)][a] = 0

        # for priority sweeping
        self.theta = theta
        self.queue = PriorityQueue()
        self.predecessors = {}  # nxtState -> list[(curState, Action)...]

    def chooseAction(self):
        # epsilon-greedy
        mx_nxt_reward = -999
        action = ""

        if np.random.uniform(0, 1) <= self.exp_rate:
            action = np.random.choice(self.actions)
        else:
            # greedy action
            current_position = self.state
            # if all actions have same value, then select randomly
            if len(set(self.Q_values[current_position].values())) == 1:
                action = np.random.choice(self.actions)
            else:
                for a in self.actions:
                    nxt_reward = self.Q_values[current_position][a]
                    if nxt_reward >= mx_nxt_reward:
                        action = a
                        mx_nxt_reward = nxt_reward
        return action

    def reset(self):
        self.maze = Maze()
        self.state = S
        self.state_actions = []

    def play(self):
        for ep in range(self.episodes):
            while not self.maze.end:

                action = self.chooseAction()
                self.state_actions.append((self.state, action))

                nxtState = self.maze.nxtPosition(action)
                reward = self.maze.giveReward() #fazer lista de recompensas

                # atualiza a fila  de prioridades
                tmp_diff = reward + np.max(list(self.Q_values[nxtState].values())) - self.Q_values[self.state][action]
                if tmp_diff > self.theta:
                    self.queue.put((-tmp_diff, (self.state, action)))  # -diff -> (state, action) pop the smallest

                # update model & predecessors
                if self.state not in self.model.keys():
                    self.model[self.state] = {}
                self.model[self.state][action] = (reward, nxtState)
                if nxtState not in self.predecessors.keys():
                    self.predecessors[nxtState] = [(self.state, action)]
                else:
                    self.predecessors[nxtState].append((self.state, action))
                self.state = nxtState

                # loop n times to randomly update Q-value
                for _ in range(self.steps):
                    if self.queue.empty():
                        break
                    _state, _action = self.queue.get()[1]
                    _reward, _nxtState = self.model[_state][_action]
                    self.Q_values[_state][_action] += self.lr * (_reward + np.max(list(self.Q_values[_nxtState].values())) - self.Q_values[_state][_action])

                    # loop for all state, action predicted lead to _state
                    if _state not in self.predecessors.keys():
                        continue
                    pre_state_action_list = self.predecessors[_state]

                    for (pre_state, pre_action) in pre_state_action_list:
                        pre_reward, _ = self.model[pre_state][pre_action]
                        pre_tmp_diff = pre_reward + np.max(list(self.Q_values[_state].values())) - self.Q_values[pre_state][pre_action]
                        if pre_tmp_diff > self.theta:
                            self.queue.put((-pre_tmp_diff, (pre_state, pre_action)))
            # end of game
            if ep % 10 == 0:
                print("episode", ep)
            self.steps_per_episode.append(len(self.state_actions))
            self.reset()

In [ ]:
N_EPISODES = 50

### Comparação

In [ ]:
import DynaMaze

ModuleNotFoundError: No module named 'DynaMaze'

In [ ]:
agent = PriorityAgent(n_steps=5, episodes=N_EPISODES)
agent.play()

steps_episode_pa = agent.steps_per_episode

In [ ]:
agent = DynaMaze.DynaAgent(n_steps=5, episodes=N_EPISODES)
agent.play()

steps_episode_da = agent.steps_per_episode

In [ ]:
plt.figure(figsize=[10, 6])

plt.ylim(0, 500)
plt.plot(range(N_EPISODES), steps_episode_pa, label="Priority Sweeping")
plt.plot(range(N_EPISODES), steps_episode_da, label="Dyna-Q")

plt.legend()